In [1]:
import MetaTrader5 as mt5
from datetime import datetime
from MyTT import *
import pandas as pd
import numpy as np
import pickle
import time
import lightgbm as lgb
import sklearn
import sys
if not mt5.initialize(login=66985382, server="BCR-MT5", password="0MxYfHq!"):
    print("initialize() failed")
    mt5.shutdown()

print(mt5.terminal_info())

def log_diff(df):
    df.replace(0, 0.01, inplace=True)
    diffdf = np.log(df).loc[1:,:] - np.log(df.shift(1)).loc[1:,:]
    return diffdf

def minicut(df, train_period=5, pred_period=1):
    '''
    train_period: 用于训练的数据长度，默认前5个交易日
    pred_period: 预测几日后的结果，默认第二天
    '''
    n_features = df.shape[1]
    roll = pd.concat([df.shift(i) for i in range(pred_period,pred_period+train_period-1)], axis=1)
    roll = pd.concat([df, roll], axis=1).dropna()
    return roll.iloc[-1:,:]

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=4330, maxbars=100000000, codepage=936, ping_last=106955, community_balance=0.0, retransmission=0.0, company='Bacera Co Pty Ltd', name='BCR MT5', language='Chinese (Simplified)', path='C:\\Program Files\\BCR MT5', data_path='C:\\Users\\YLfan\\AppData\\Roaming\\MetaQuotes\\Terminal\\ACB9E6E50A3004D5DCBF6CD96E6D7AC5', commondata_path='C:\\Users\\YLfan\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')


In [2]:
mt5.account_info()._asdict()

{'login': 66985382,
 'trade_mode': 0,
 'leverage': 200,
 'limit_orders': 200,
 'margin_so_mode': 0,
 'trade_allowed': True,
 'trade_expert': True,
 'margin_mode': 2,
 'currency_digits': 2,
 'fifo_close': False,
 'balance': 14690.85,
 'credit': 0.0,
 'profit': 0.0,
 'equity': 14690.85,
 'margin': 0.0,
 'margin_free': 14690.85,
 'margin_level': 0.0,
 'margin_so_call': 150.0,
 'margin_so_so': 50.0,
 'margin_initial': 0.0,
 'margin_maintenance': 0.0,
 'assets': 0.0,
 'liabilities': 0.0,
 'commission_blocked': 0.0,
 'name': 'Xu YuFan',
 'server': 'BCR-MT5',
 'currency': 'USD',
 'company': 'Bacera Co Pty Ltd'}

# Demo

In [5]:
maximum = 1
lot = 0.4
emphsize = 3.3
deviation = 10
tick_time = 0
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M5
point = mt5.symbol_info(symbol).point
model = pickle.load(open('model/EURUSD_M5_10_1.pickle', 'rb')).booster_

try:
    while True:
        # 检测持仓
        accInfo = mt5.account_info()._asdict()

        if accInfo['equity']/maximum < 0.9: #回撤10%，强制平仓
            for position in mt5.positions_get():
                position = position._asdict()
                mt5.Close(position['ticket'])
            sys.exit(1)
        else:
            maximum = max(accInfo['equity'], maximum)

        LastBars = mt5.copy_rates_from_pos(symbol, timeframe, 0, 100)
        if LastBars[-1][0] > tick_time:
            tick_time = LastBars[-1][0]
            if accInfo['margin_free'] > accInfo['balance'] * 0.6: # margin超过balance六成，可以下单
                x = pd.DataFrame(LastBars).iloc[:-1,[1,2,3,4,5]]
                x['MA5'] = MA(x['close'], 5)
                x['MA15'] = MA(x['close'], 15)
                x['MA30'] = MA(x['close'], 30)
                x['MA60'] = MA(x['close'], 60)
                x['vMA5'] = MA(x['tick_volume'], 5)
                x['vMA15'] = MA(x['tick_volume'], 15)
                x['vMA30'] = MA(x['tick_volume'], 30)
                x['vMA60'] = MA(x['tick_volume'], 60)
                X = np.ascontiguousarray(minicut(log_diff(x),10,1))
                pchg = (np.exp(model.predict(X)[0])-1) * emphsize / point
                print(time.ctime(), pchg, end=' ')
                sl = round(LastBars[-2][-4]*(1-10*pchg*point), 5)
                tp = round(LastBars[-2][-4]*(1+5*pchg*point), 5)
                LastTick = mt5.symbol_info_tick(symbol)
                if  pchg > 1 and tp > LastTick.ask:
                    request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "symbol": symbol,
                        "volume": lot,
                        "type": mt5.ORDER_TYPE_BUY,
                        "price": LastTick.ask,
                        "sl": sl,
                        "tp": tp,
                        "deviation": deviation,
                        "magic": 224700,
                        "type_time": mt5.ORDER_TIME_GTC,
                        "type_filling": mt5.ORDER_FILLING_FOK,
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(result.order, 'buy', lot, end=' ')
                elif pchg < -1 and tp < LastTick.bid:
                    request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "symbol": symbol,
                        "volume": lot,
                        "type": mt5.ORDER_TYPE_SELL,
                        "price": LastTick.bid,
                        "sl": sl,
                        "tp": tp,
                        "deviation": deviation,
                        "magic": 224700,
                        "type_time": mt5.ORDER_TIME_GTC,
                        "type_filling": mt5.ORDER_FILLING_FOK,
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(result.order, 'sell', lot, end=' ')
                print()
except SystemExit:
    for position in mt5.positions_get():
        position = position._asdict()
        mt5.Close(position['symbol'], ticket=position['ticket'])
    print('清仓')

Tue Jun  4 02:56:50 2024 -4.770970514179628 478965 sell 0.4 
Tue Jun  4 02:59:59 2024 -0.7287991196269771 
Tue Jun  4 03:04:59 2024 -2.132888824574186 
Tue Jun  4 03:09:59 2024 -3.4032432455211032 479134 sell 0.4 
Tue Jun  4 03:14:59 2024 3.144949109314954 479160 buy 0.4 
Tue Jun  4 03:19:59 2024 -4.787030342103814 479174 sell 0.4 
Tue Jun  4 03:24:59 2024 0.6642882918272973 
Tue Jun  4 03:29:59 2024 1.9489695329077956 
Tue Jun  4 03:35:01 2024 -0.3469866413929878 
Tue Jun  4 03:40:00 2024 -1.9932667646405107 
Tue Jun  4 03:45:07 2024 -4.915229738915449 479253 sell 0.4 
Tue Jun  4 03:50:05 2024 -6.724700201491405 479283 sell 0.4 
Tue Jun  4 03:54:59 2024 -3.7009515507158133 479324 sell 0.4 
Tue Jun  4 03:59:59 2024 -2.249092573660682 
Tue Jun  4 04:05:04 2024 0.9242434710365898 
Tue Jun  4 04:09:59 2024 -1.813070960312579 
Tue Jun  4 04:14:59 2024 -1.237351810479126 
Tue Jun  4 04:19:59 2024 -5.769453167694393 479345 sell 0.4 
Tue Jun  4 04:24:59 2024 -4.455140839469961 479352 sell 0.4

AttributeError: 'NoneType' object has no attribute '_asdict'